In [1]:
import sys
sys.path.append('..')

In [2]:
import os
os.environ['POSTGRES_USER'] =  'postgres'
os.environ['POSTGRES_PASSWORD'] = 'postgres'
os.environ['POSTGRES_HOST'] = 'localhost:5432'
os.environ['POSTGRES_DB'] = 'postgres'

In [3]:
from requests import get, post

In [4]:
from src.protocols.dexes.amms.utils import get_mint_decimals
from solana.rpc.async_api import AsyncClient
from solders.pubkey import Pubkey

In [5]:
from solana.rpc.async_api import AsyncClient
from solders.pubkey import Pubkey

In [6]:
from src.protocols.anchor_clients.marginfi_client.accounts import Bank, MarginfiGroup
from decimal import Decimal

In [14]:
from src.protocols.dexes.amms.utils import get_tokens_address_to_info_map

In [15]:
AUTHENTICATED_RPC_URL = ""
client = AsyncClient(AUTHENTICATED_RPC_URL)
tokens = get_tokens_address_to_info_map()

# Marginfi

In [10]:
def get_banks():
    banks = get('https://storage.googleapis.com/mrgn-public/mrgn-bank-metadata-cache.json').json()
    return banks

In [11]:
banks = get_banks()

In [12]:
banks[0]

{'bankAddress': '9dpu8KL5ABYiD3WP2Cnajzg1XaotcJvZspv29Y1Y3tn1',
 'tokenAddress': 'EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm',
 'tokenName': 'dogwifhat',
 'tokenSymbol': '$WIF'}

6

In [20]:
# Fethcing 
b = await Bank.fetch(client, Pubkey.from_string('9dpu8KL5ABYiD3WP2Cnajzg1XaotcJvZspv29Y1Y3tn1'))
mint_decimals = tokens[str(b.mint)]['decimals']

# liq_v = await client.get_account_info_json_parsed(b.liquidity_vault)
# fee_v = await client.get_account_info_json_parsed(b.fee_vault)
# fees = Decimal(b.collected_group_fees_outstanding.value) / 2**48
# insurance = Decimal(b.collected_insurance_fees_outstanding.value) / 2**48

asset_shares = Decimal(b.total_asset_shares.value) / 2**48
asset_share_value = Decimal(b.asset_share_value.value) / 2**48
assets = (asset_shares * asset_share_value)

liability_shares = Decimal(b.total_liability_shares.value) / 2**48
liability_share_value = Decimal(b.liability_share_value.value) / 2**48
liabilities = (liability_shares * liability_share_value)

# Total Available from https://app.marginfi.com
min(
    (assets - liabilities) / 10**mint_decimals,
    (b.config.borrow_limit - liabilities) / 10**mint_decimals
)

Decimal('1284.153759511931515383626816')

In [21]:
# Marginfi
print(Decimal(b.config.liability_weight_init.value) / 2**48)
print(Decimal(b.config.liability_weight_maint.value) / 2**48)

print(Decimal(b.config.asset_weight_init.value) / 2**48)
print(Decimal(b.config.asset_weight_maint.value) / 2**48)


2.5
1.5
0
0


In [ ]:
# timestamp = Column(BigInteger, nullable = False)
#     protocol_id = Column(String, nullable = False)
#     market = Column(String)
#     vault = Column(String, nullable = False)
#     underlying_mint_address = Column(String, nullable = False)
#     deposits_total = Column(Numeric, nullable = False)
#     lent_total = Column(Numeric, nullable = False)
#     available_to_borrow = Column(Numeric, nullable = False)

# Kamino

In [34]:
from src.protocols.anchor_clients.kamino_client.accounts.lending_market import LendingMarket
from src.protocols.anchor_clients.kamino_client.accounts.reserve import Reserve

In [35]:
r = get('https://api.hubbleprotocol.io/v2/kamino-market')

In [36]:
r.json()

[{'lendingMarket': '7u3HeHxYDLhnCoErrtycNokbQYbWGzLs6JSDqGAv5PfF',
  'isPrimary': True,
  'name': 'Primary market on mainnet',
  'description': 'Main pool',
  'lookupTable': '284iwGtA9X9aLy3KsyV8uT2pXLARhYbiSi5SiM2g47M2'},
 {'lendingMarket': 'DxXdAyU3kCjnyggvHmY5nAwg5cRbbmdyX3npfDMjjMek',
  'isPrimary': False,
  'name': 'Isolated JLP market on mainnet',
  'description': 'Isolated pool',
  'lookupTable': 'GprZNyWk67655JhX6Rq9KoebQ6WkQYRhATWzkx2P2LNc'},
 {'lendingMarket': 'ByYiZxp8QrdN9qbdtaAiePN8AAr3qvTPppNJDpf5DVJ5',
  'isPrimary': False,
  'name': 'Solana ecosystem market on mainnet',
  'description': 'Ecosystem pool',
  'lookupTable': 'x2uEQSaqrZs5UnyXjiNktRhrAy6iNFeSKai9VNYFFuy'}]

In [37]:
kamino_markets = [
    {
        'address': Pubkey.from_string(i['lendingMarket']),
        'market': await LendingMarket.fetch(client, Pubkey.from_string(i['lendingMarket']))
    }
    for i in r.json()
]

In [38]:
# kamino_markets

In [39]:
from solana.rpc.types import MemcmpOpts

In [42]:
PROGRAM_ID = Pubkey.from_string("KLend2g3cP87fffoy8q1mQqGKjrxjC8boSyAYavgmjD")

In [71]:
# accs = await client.get_program_accounts(
#     PROGRAM_ID,
#     encoding='base64',
#     filters = [Reserve.layout.sizeof() + 8, MemcmpOpts(32, str(main))]
# )

In [45]:
for market in kamino_markets:
    filters = [
        Reserve.layout.sizeof() + 8,
        MemcmpOpts(32, str(market['address']))
    ]
    
    market_accounts = await client.get_program_accounts(
        PROGRAM_ID,
        encoding='base64',
        filters = filters
    )   

    market_reserves = [
        {'address': i.pubkey, 'info': Reserve.decode(i.account.data)}
        for i in market_accounts.value
    ]

    market['reserves'] = market_reserves

In [55]:
t = kamino_markets[0]['reserves'][2]

In [56]:
t['info'].liquidity.mint_pubkey

Pubkey(
    So11111111111111111111111111111111111111112,
)

In [ ]:
t

In [ ]:
@dataclass
class Vault:
    address: Pubkey
    market: str | None
    mint_address: Pubkey
    deposits: Decimal
    lent: Decimal
    available_to_borrow: Decimal


In [155]:
vaults = []

for market in kamino_markets:
    for reserve in market['reserves']:
        available_assets = Decimal(reserve['info'].liquidity.available_amount)
        liabilities = Decimal(reserve['info'].liquidity.borrowed_amount_sf) / 2**60
        assets = available_assets + liabilities
        total_available = min(
            assets - liabilities,
            reserve['info'].config.borrow_limit - liabilities
        )   

        v = Vault(
            address = reserve['address'],
            market = market['address'],
            mint_address = reserve['info'].liquidity.mint_pubkey,
            deposits = assets,
            lent = liabilities,
            available_to_borrow=total_available
        )
        vaults.append(v)
    

In [157]:
vaults[0]

Vault(address=Pubkey(
    febGYTnFX4GbSGoFHFeJXUHgNaK53fB23uDins9Jp1E,
), market='7u3HeHxYDLhnCoErrtycNokbQYbWGzLs6JSDqGAv5PfF', mint_address=Pubkey(
    7vfCXTUXx5WJV5JADk17DUJ4ksgau7utNKj4b963voxs,
), deposits=Decimal('54556403399.27403226293838483'), lent=Decimal('19999057165.27403226293838483'), available_to_borrow=Decimal('942834.72596773706161517'))

In [151]:
kamino_markets[0]['reserves'][0]['info'].liquidity.mint_pubkey

Pubkey(
    7vfCXTUXx5WJV5JADk17DUJ4ksgau7utNKj4b963voxs,
)

In [112]:
f"{t.liquidity.available_amount / 10**9:_}"

'848_967.45879326'

In [130]:
available_assets = Decimal(t.liquidity.available_amount)
liabilities = Decimal(t.liquidity.borrowed_amount_sf) / 2**60
assets = available_assets + liabilities

In [120]:
t.config.borrow_limit / 10**9

2400000.0

In [131]:
total_available = min(
    assets - liabilities,
    t.config.borrow_limit - liabilities
)   


# Solend
- API: https://api.solend.fi/#/
- TVL calculator: https://github.com/solendprotocol/tvl-calculator-example
- Docs: https://docs.solend.fi/developers/integration-guide
- Repo: https://github.com/solendprotocol/public
- API: https://app.swaggerhub.com/apis-docs/solendprotocol/API/1.0.1#/Reserves/get_v1_config_

In [10]:
RESERVES_TO_ASSET_MAP = {
  "8PbodeaosQP19SjYFx855UMqWxH2HynZLdBXmsrbac36": "SOL",
  "BgxfHJDzm44T7XG68MYKx7YisTjZu73tVovyZSjJMpmw": "USDC",
  "3PArRsZQ6SLkr1WERZWyC6AqsajtALMq4C66ZMYz4dKQ": "ETH",
  "GYzjMCXTDue12eUGKKWAqtF5jcBYNmewr6Db6LaguEaX": "BTC",
}

In [11]:
SOLEND_PROGRAM_ID = Pubkey.from_string("So1endDq2YkqhipRh3WViPa8hdiSpxWy6z3Z6tMCpAo")
LENDING_MARKET_MAIN = Pubkey.from_string("4UpD2fh7xH3VP9QQaXtsS1YY3bxzWhtfpks7FatyKvdY")
OBLIGATION_LEN = 1300

In [14]:
res = await client.get_program_accounts(
    SOLEND_PROGRAM_ID, 
    encoding='base64',
    filters=[
        OBLIGATION_LEN, MemcmpOpts(offset=10, bytes = str(LENDING_MARKET_MAIN))
    ]
)

In [22]:
import borsh_construct as borsh

In [ ]:
# layout: typing.ClassVar = borsh.CStruct(
#     "version" / borsh.U64,
#     "last_update" / types.last_update.LastUpdate.layout,
#     "lending_market" / BorshPubkey,
#     "farm_collateral" / BorshPubkey,
#     "farm_debt" / BorshPubkey,
#     "liquidity" / types.reserve_liquidity.ReserveLiquidity.layout,
#     "reserve_liquidity_padding" / borsh.U64[150],
#     "collateral" / types.reserve_collateral.ReserveCollateral.layout,
#     "reserve_collateral_padding" / borsh.U64[150],
#     "config" / types.reserve_config.ReserveConfig.layout,
#     "config_padding" / borsh.U64[150],
#     "padding" / borsh.U64[240],
# )

In [24]:
from anchorpy.borsh_extension import BorshPubkey

In [23]:
LastUpdateLayout = borsh.CStruct(
    "slot" / borsh.U64, 
    "stale" / borsh.U8
)

In [54]:
ObligationLayout = borsh.CStruct(
    "version" / borsh.U8,
    "last_update" / LastUpdateLayout,
    "lendingMarket" / BorshPubkey,
    "owner" / BorshPubkey,
    "depositedValue" / borsh.U128,
    "borrowedValue" / borsh.U128,
    "allowedBorrowValue" / borsh.U128,
    "unhealthyBorrowValue" / borsh.U128,
    "depositsLen" / borsh.U8,
    "borrowsLen" / borsh.U8,
    # "dataFlat" / borsh.U64[1096],
    "padding" / borsh.U64[64],
)

In [1]:
ObligationLayout

NameError: name 'ObligationLayout' is not defined

In [55]:
t = ObligationLayout.parse(res.value[2].account.data)

In [59]:
t

Container(version=1, last_update=Container(slot=97615694, stale=1), lendingMarket=Pubkey(
    4UpD2fh7xH3VP9QQaXtsS1YY3bxzWhtfpks7FatyKvdY,
), owner=Pubkey(
    DCmAa5rdFMTfxVjnuUUCunnvF7MLwGEMogUvqLEy19cB,
), depositedValue=5001000281814372544351, borrowedValue=0, allowedBorrowValue=3750750211360779408263, unhealthyBorrowValue=4000800225451498035480, depositsLen=0, borrowsLen=0, padding=ListContainer([0, 0, 0, 0, 0, 0, 0, 0, 7434955088037466526, 1444905396176572961, 10349855295385364050, 8833716947689890378, 4971345166, 1932637839084056415, 271, 0, 0, 0, 0, 7434955088037466526, 1444905396176572961, 10349855295385364050, 8833716947689890378, 1008097203541760177, 0, 2637857927020440704, 10843106, 15592567247663684726, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [59]:
base = "https://api.solend.fi"

In [60]:
r = get(base + "/v1/markets/configs?scope=all&deployment=production")

In [63]:
r.json()[0]['reserves'][0]

{'liquidityToken': {'coingeckoID': 'tbtc',
  'decimals': 8,
  'logo': 'https://assets.coingecko.com/coins/images/11224/large/0x18084fba666a33d37592fa2633fd49a74dd93a88.png',
  'mint': '6DNSN2BJsaPFdFFc1zP37kkeNe4Usc1Sqkzr9C9vPWcU',
  'name': 'tBTC',
  'symbol': 'tBTC',
  'volume24h': '0'},
 'pythOracle': 'GVXRSBjFk6e6J3NbVPXohDJetcTjaeeuykUpbQF8UoMU',
 'switchboardOracle': 'nu11111111111111111111111111111111111111111',
 'address': '3B29sQsgsVuthQurNux82vxN6N3AVKo3zArPQSzoe3jN',
 'collateralMintAddress': 'JDMW7AooL6qVAfXdRQytwW9UvR4n6EvqpjQBpzabDwf8',
 'collateralSupplyAddress': '4wKgt6WHsR8f8yU1AU74yYyPqvjHHXwga3Ed1pKz9xd4',
 'liquidityAddress': '23YKkRwW5wvE3zNmQkzQm8eCX6TZkaHroyELtNE371CP',
 'liquidityFeeReceiverAddress': 'DEo2AaopAsCV9Ek3cYo1pFoLUX6Bdy3Cs8F2xuuSMDPY'}

In [57]:
r.json()['results']

TypeError: list indices must be integers or slices, not str

In [66]:
shit = get(base + '/v1/reserves?scope=solend&ids=3B29sQsgsVuthQurNux82vxN6N3AVKo3zArPQSzoe3jN')

In [69]:
shit.json()['results']

[{'reserve': {'version': 1,
   'lastUpdate': {'slot': '260227884', 'stale': 0},
   'lendingMarket': '4UpD2fh7xH3VP9QQaXtsS1YY3bxzWhtfpks7FatyKvdY',
   'liquidity': {'mintPubkey': '6DNSN2BJsaPFdFFc1zP37kkeNe4Usc1Sqkzr9C9vPWcU',
    'mintDecimals': 8,
    'supplyPubkey': '23YKkRwW5wvE3zNmQkzQm8eCX6TZkaHroyELtNE371CP',
    'oracleOption': None,
    'pythOracle': 'GVXRSBjFk6e6J3NbVPXohDJetcTjaeeuykUpbQF8UoMU',
    'switchboardOracle': 'nu11111111111111111111111111111111111111111',
    'availableAmount': '16910511',
    'borrowedAmountWads': '2518290157591039402552839',
    'cumulativeBorrowRateWads': '1003462290270906042',
    'marketPrice': '66429354066900000000000',
    'accumulatedProtocolFeesWads': '83492712272290034542',
    'smoothedMarketPrice': None},
   'collateral': {'mintPubkey': 'JDMW7AooL6qVAfXdRQytwW9UvR4n6EvqpjQBpzabDwf8',
    'mintTotalSupply': '19423357',
    'supplyPubkey': '4wKgt6WHsR8f8yU1AU74yYyPqvjHHXwga3Ed1pKz9xd4'},
   'config': {'optimalUtilizationRate': 80,
    'm

In [119]:
t =r.json()['results'][0]

In [120]:
t.keys()

dict_keys(['reserve', 'cTokenExchangeRate', 'rates', 'rewards'])

In [121]:
t['cTokenExchangeRate']

'1.13574416684194515394'

In [125]:
t['reserve']['liquidity']

{'mintPubkey': 'So11111111111111111111111111111111111111112',
 'mintDecimals': 9,
 'supplyPubkey': '8UviNr47S8eL6J3WfDxMRa3hvLta1VDJwNWqsDgtN3Cv',
 'oracleOption': None,
 'pythOracle': 'H6ARHf6YXhGYeQfUzQNGk6rDNnLBQKrenN712K4AQJEG',
 'switchboardOracle': 'GvDMxPzN1sCj7L26YDK2HnMRXEQmQ2aemov8YBtPS7vR',
 'availableAmount': '307364551705498',
 'borrowedAmountWads': '118967870226535547286558439168916',
 'cumulativeBorrowRateWads': '1225272605901833295',
 'marketPrice': '144955001490000000000',
 'accumulatedProtocolFeesWads': '4188813419086268323467717161',
 'smoothedMarketPrice': None}

# Mango

In [71]:
from src.protocols.anchor_clients.mango_client.accounts.group import Group
from src.protocols.anchor_clients.mango_client.accounts.bank import Bank
from src.protocols.anchor_clients.mango_client.accounts.perp_market import PerpMarket

In [70]:
r = get('https://api.mngo.cloud/data/v4/group-metadata').json()['groups']

In [72]:
_group = r[0]
group = await Group.fetch(client, Pubkey.from_string('78b8f4cGCwmZ9ysPFMWLaLTkkaYnUjwMJYStWe5RTSSX'))

In [ ]:
banks = group

In [75]:
_group['tokens']

[{'group': '78b8f4cGCwmZ9ysPFMWLaLTkkaYnUjwMJYStWe5RTSSX',
  'mint': 'JUPyiwrYJFskUPiHa7hkeR8VUtAeFoSYbKedZNsDvCN',
  'tokenIndex': 894,
  'symbol': 'JUP',
  'decimals': 6,
  'oracle': 'ARXKK2gw7zpjonHbmcqYj7pQzwAxAa49Yy7kh5P44Aek',
  'mintInfo': '3PLpM8qWM8jsFBt2zREydfjC58rhPqiJV6LbQRywjHwA',
  'banks': [{'bankNum': 0,
    'publicKey': 'BqHd7o8aHSmx6LtsV6onYQL5TbDrnXHabXkVXT4aRCfZ'}],
  'active': True},
 {'group': '78b8f4cGCwmZ9ysPFMWLaLTkkaYnUjwMJYStWe5RTSSX',
  'mint': '45EgCwcPXYagBC7KqBin4nCFgEZWN7f3Y6nACwxqMCWX',
  'tokenIndex': 889,
  'symbol': 'Moutai',
  'decimals': 6,
  'oracle': 'AV67ufGVkHrPKXdeupXE2MXdw3puq7xnkPNrTxGP3suU',
  'mintInfo': '66B1a7RF9J9sAAnsBVMjSYTELKEx934eNk4CKdwLwEMZ',
  'banks': [{'bankNum': 0,
    'publicKey': '7W12TvXN4yoUwbYTRwMj754EvkFsZtX25MzvxSjeHxSL'}],
  'active': True},
 {'group': '78b8f4cGCwmZ9ysPFMWLaLTkkaYnUjwMJYStWe5RTSSX',
  'mint': 'BLZEEuZUBVqFhj8adcCFPJvPVCiCyVmh3hkJMrU8KuJA',
  'tokenIndex': 891,
  'symbol': 'BLZE',
  'decimals': 9,
  'or

In [145]:
banks = list(map(
    lambda x: {'name': x['symbol'], 'mint': x['mint'], 'bankNum': x['banks'][0]['bankNum'], 'pubkey': x['banks'][0]['publicKey'], 'active': x['active']},
    _group['tokens']
))

In [153]:
banks = list(filter(lambda x: x['active'], banks))

In [76]:
ba = await Bank.fetch(client, Pubkey.from_string('FqEhSJSP3ao8RwRSekaAQ9sNQBSANhfb6EPtxQBByyh5'))

In [86]:
# Mango
print(Decimal(ba.init_asset_weight.val) / 2**48)
print(Decimal(ba.maint_asset_weight.val) / 2**48)

print(Decimal(ba.init_liab_weight.val) / 2**48)
print(Decimal(ba.maint_liab_weight.val) /2**48)

0.800000011920928955078125
0.89999997615814208984375
1.2000000476837158203125
1.10000002384185791015625


In [77]:
ba

Bank(group=Pubkey(
    78b8f4cGCwmZ9ysPFMWLaLTkkaYnUjwMJYStWe5RTSSX,
), name=ListContainer([83, 79, 76, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), mint=Pubkey(
    So11111111111111111111111111111111111111112,
), vault=Pubkey(
    6NfwhGjfrE4dp7XFut2kYcUQsH4P4AJVSoe9cLVM2z43,
), oracle=Pubkey(
    H6ARHf6YXhGYeQfUzQNGk6rDNnLBQKrenN712K4AQJEG,
), oracle_config=OracleConfig(conf_filter=I80F48(val=28147498090496), max_staleness_slots=180, reserved=ListContainer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), stable_price_model=StablePriceModel(stable_price=0.15217284483195925, last_update_timestamp=1713179743, delay_prices=ListContainer([0.14971688985428025, 0.1483727446354589, 0.14753561784022434, 0.14845320121694283, 0.14784503424093062, 0.14955007088639183, 0.15397963198725842, 0.155005496474633, 0.1541891328481755, 0.1

In [99]:
# [i for i in r[0]['tokens'] if 'sol' in i['symbol'].lower()][1]

In [102]:
ba.collected_fees_native.val 

784095079695.3711

In [117]:
round(ba.liquidation_fee.val / 2**48, 2)

0.01

In [120]:
ba.net_borrow_limit_per_window_quote /10**9

1000.0

In [75]:
r[0]

{'publicKey': '78b8f4cGCwmZ9ysPFMWLaLTkkaYnUjwMJYStWe5RTSSX',
 'cluster': 'MAINNET',
 'name': 'MAINNET.0',
 'mangoProgramId': '4MangoMjqJ2firMokCjjGgoK8d4MXcrgL7XJaL3w6fVg',
 'insuranceVault': 'F1vqFqkZHh5jd5rf8BcEzT8kqfd1snB1adRkayJ9KPNY',
 'insuranceMint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
 'insuranceMintDecimals': 6,
 'perpMarkets': [{'group': '78b8f4cGCwmZ9ysPFMWLaLTkkaYnUjwMJYStWe5RTSSX',
   'publicKey': 'HwhVGkfsSQ9JSQeQYu2CbkRCLvsh3qRZxG6m4oMVwZpN',
   'marketIndex': 0,
   'name': 'BTC-PERP',
   'baseDecimals': 6,
   'baseLotSize': 100,
   'quoteLotSize': 10,
   'oracle': 'GVXRSBjFk6e6J3NbVPXohDJetcTjaeeuykUpbQF8UoMU',
   'active': True,
   'settleTokenIndex': 0},
  {'group': '78b8f4cGCwmZ9ysPFMWLaLTkkaYnUjwMJYStWe5RTSSX',
   'publicKey': 'GcMimCLCU8aQhUpZwB5dWTQDxkTzuMy8uKQfujjYjz4b',
   'marketIndex': 4,
   'name': 'RENDER-PERP',
   'baseDecimals': 6,
   'baseLotSize': 100000,
   'quoteLotSize': 10,
   'oracle': 'CYGfrBJB9HgLf9iZyN4aH5HvUAi2htQ4MjPxeXMf4Egn',
   

# Calculating liquidity - intra range
- https://uniswapv3book.com/milestone_1/calculating-liquidity.html#price-range-calculation

- Buying ETH for USDC @ 5000
- Current tick 5k, lower/upper bound: (4545, 5500)
  
- $\sqrt{P} =  \sqrt{\frac{y}{x}}$

- ETH is $x$, USDC is $y$
  - $\sqrt{P_c} = \sqrt{\frac{5000}{1}} = \sqrt{5000} \approx 70.71$
  - $\sqrt{P_l} = \sqrt{\frac{4545}{1}} \approx 67.42$
  - $\sqrt{P_u} = \sqrt{\frac{5500}{1}} \approx 74.16$

- Prices and ticks connected: $\sqrt{P(i)} = 1.0001^{\frac{i}{2}}$
- We can find tick $i$ with: $i = log_{\sqrt{1.0001}}\sqrt{P(i)}$

- $i_c = 85176$
- $i_l = 84222$
- $i_u = 86129$

In [ ]:
import math

def uni_price_to_tick(p):
    return math.floor(math.log(p, 1.0001))

uni_price_to_tick(5000)

In [ ]:
# Uniswap uses Q64.96 - so we need to multiply numbers by 2**96
q96 = 2**96

def uni_price_to_sqrtp(p):
    return int(math.sqrt(p) * q96)

uni_price_to_sqrtp(5000)

### Liquidity amount calculation
- Need to calculate $L$ based on the amount we'll deposit
- For inifinite curve: $L = \sqrt{xy}$
- But we need to calculate $L$ for the specific price range
    - Need to find $L$ that will allow the price to move to either points
    - $L$ must be calculated on the maximum amount for $\Delta x$ and $\Delta y$
- The curve is split into two segment - each of the tokens contributes to one
  - Left segment will be composed of $x$, right of $y$

- When providing liq we calculate two $L$'s and pick one - the smaller one (because the bigger one already includes the smaller one) - we want to add the same $L$ to the left and right

- $\Delta x = \Delta \frac{1}{\sqrt{P}}L$
- $\Delta y = \Delta \sqrt{P}L$
- We can expand these by replace the deltas with actuals:
$$
\Delta x = (\frac{1}{\sqrt{P_c}} - \frac{1}{\sqrt{P_b}})L
$$
$$
\Delta y = (\sqrt{P_c} - \sqrt{P_a})L
$$
- $P_a$ - price at point a
- $P_b$ - price at point b

- $L$ from first formula (derivation skipped): $L = \Delta x \frac{\sqrt{P_b}\sqrt{P_c}}{\sqrt{P_c} - \sqrt{P_a}}$
- $L$ from the second formula: $L = \frac{\Delta y}{\sqrt{P_c} - \sqrt{P_a}}$

In [ ]:
sqrtp_low = uni_price_to_sqrtp(4545)
sqrtp_cur = uni_price_to_sqrtp(5000)
sqrtp_upp = uni_price_to_sqrtp(5500)


def liquidity0(amount, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return (amount * (pa * pb) / q96) / (pb - pa)

def liquidity1(amount, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return amount * q96 / (pb - pa)

eth = 10**18
amount_eth = 1 * eth
amount_usdc = 5000 * eth

liq0 = liquidity0(amount_eth, sqrtp_cur, sqrtp_upp)
liq1 = liquidity1(amount_usdc, sqrtp_cur, sqrtp_low)
liq = int(min(liq0, liq1))
liq

In [ ]:
# Amounts to be deposited
def calc_amount0(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * q96 * (pb - pa) / pa / pb)


def calc_amount1(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * (pb - pa) / q96)

amount0 = calc_amount0(liq, sqrtp_upp, sqrtp_cur)
amount1 = calc_amount1(liq, sqrtp_low, sqrtp_cur)
(amount0, amount1) 

## Swap - within a price range
- ie we want to swap 42 USDC for ETH

- Only sqrt price changes 
- V3 acts like a v2 when swapping within price range

- $L = \frac{\Delta y}{\Delta \sqrt{P}}$
- We know $\Delta y$ - 42 USDC 
- How selling 42 USDC will affect the sqrt price: $\Delta \sqrt{P} = \frac{\Delta y}{L}$
- $\Delta \sqrt{P} = \frac{42 USDC}{1517882343751509868544} = 2192253463713690532467206957$
- $\sqrt{P_{target}} = sqrt{P_{current}} + \Delta \sqrt{P} = 5604469350942327889444743441197$

In [ ]:
amount_in = 42 * eth
price_diff = (amount_in * q96) // liq
price_next = sqrtp_cur + price_diff

print("New price:", (price_next / q96) ** 2)
print("New sqrtP:", price_next)
print("New tick:", price_to_tick((price_next / q96) ** 2))

In [ ]:
amount_in = calc_amount1(liq, price_next, sqrtp_cur)
amount_out = calc_amount0(liq, price_next, sqrtp_cur)

print("USDC in:", amount_in / eth)
print("ETH out:", amount_out / eth)

- We know that:
-  $\Delta x = \Delta \frac{1}{\sqrt{P}}L$
-  $\Delta \frac{1}{\sqrt{P}} = \frac{1}{\sqrt{P_{target}}} - \frac{1}{\sqrt{P_{current}}}$

In [ ]:
((((1/price_next) - (1/sqrtp_cur)) * q96) * liq) / eth # Negative since ETH gets taken out

## Selling token $x$

In [ ]:
a = [10]

In [ ]:

if not (b := [i for i in a if i > 0]):
    print('nope')

In [ ]:
b